In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp

# Parameters
g = 1  # Recovery rate - dimensionless
b = 10  # Infection rate - dimensionless
a = 0.01  # Initial percentage of population that is infected
R0 = b / g  # Reproduction number
eps = 1 / R0  # Small parameter: 1/R0

t_star = -eps * np.log(a) / g  # time at which the late time solution can be used from

# Functions
def r_late(t):
    return (1 - np.exp(-g * (t - t_star))) * (t > t_star)

def r_initial(t):
    return eps * np.log(1 - a + a * np.exp(g * t / eps))

def r_composite(t):
    return r_initial(t) + (r_late(t) - (eps * np.log(a) + g * t)) * (t > t_star)

def i(r):
    return -(1 - a) * np.exp(-R0 * r) + 1 - r

# ODE function for numerical solution
def recovered_ode(t, r):
    return g * (1 - r - (1 - a) * np.exp(-r / eps))

# Solve the ODE numerically
sol = solve_ivp(recovered_ode, [0, 3], [0], t_eval=np.linspace(0, 3, 300))

t = sol.t
r_num = sol.y[0]

# Plotting
plt.figure(figsize=(14, 6))

# Initial and late solutions
plt.subplot(1, 2, 1)
plt.plot(t, r_initial(t), label='Initial solution')
plt.plot(t, r_late(t), label='Late solution')
plt.plot(t, i(r_composite(t)), label='Composite solution')
plt.plot(t, r_num, label='Numerical solution')
plt.ylim([-0.1, 1.1])
plt.xlabel('t')
plt.ylabel('Recovered population, r(t)')
plt.legend()

# Composite and numerical solutions
plt.subplot(1, 2, 2)
plt.plot(t, r_num, 'r', label='r(t) Numerical solution')
plt.plot(t, r_composite(t), 'r-.', label='r(t) Composite solution')
plt.plot(t, i(r_num), 'k-.', label='i(t) - Numerical')
plt.plot(t, i(r_composite(t)), 'k', label='i(t) Composite solution')
plt.xlabel('t')
plt.ylabel('Population percentage')
plt.legend()

plt.tight_layout()
plt.show()
